In [1]:
#Data
#Trade Logic
#Instrument
#TradeLog

#Metrics Analysis

In [2]:
import math
import requests
import datetime
import numpy as np
import pandas as pd
import re
from glob import glob

In [12]:
all_files = pd.DataFrame(glob('/Users/yashraj/Documents/QL/QL Data/banknifty/*'),columns=['location'])
all_files['data_date'] = pd.to_datetime(all_files['location'].apply(lambda x: re.findall(r'[0-9]{4}-[0-9]{2}-[0-9]{2}',x)[0]))
all_files.sort_values('data_date',inplace=True)

In [14]:
all_files.reset_index(drop=True,inplace=True)

In [16]:
system_type = 'short'

In [17]:
entry_offset = 'close'

if system_type == 'short':
    stop_loss_offset = 'high'
elif system_type == 'long':
    stop_loss_offset = 'low'

weekday_dict = {0:'MONDAY',
                1:'TUESDAY',
                2:'WEDNESDAY',
                3:'THURSDAY',
                4:'FRIDAY',
                5:'SATURDAY'}

In [19]:
entry_time = datetime.time(9,20)
exit_time = datetime.time(15,14)
stop_loss_percentage = 0.40
instrument = 'BANKNIFTY'
base = 100
option_value = 100

In [89]:
trade_log = pd.DataFrame(columns=['entry_date','days_to_expiry','stop_loss','ce_entry_time','ce_symbol','ce_entry','ce_exit','ce_exit_time','ce_pnl','pe_entry_time','pe_symbol','pe_entry','pe_exit','pe_exit_time','pe_pnl'])

for index, row in all_files[495:].iterrows():

    data = pd.read_parquet(row['location'])
    data.sort_values('datetime',inplace=True)
    data_date = row['data_date'].date()
    data['strike_price'] = data['strike_price'].apply(float)
    
    print(f'FETCHED DATA FOR {data_date}')

    ce_strike_alternatives = 0
    pe_strike_alternatives = 0
    entry_minutes_alternative = 1

    entry_datetime = datetime.datetime.combine(data_date,entry_time)
    exit_datetime = datetime.datetime.combine(data_date,exit_time)

    #futures_nearest_expiry = data[(data['instrument_type'] == 'FUT') & (data['expiry_date'] >= data_date)]['expiry_date'].min()
        
    #subset the futures data
    futures_data = data[(data['instrument_type'] == 'FUT') & (data['expiry_type'] == 'I')].copy()
    futures_data.drop_duplicates(subset=['datetime'],keep='first',inplace=True)
    futures_data.reset_index(drop=True, inplace=True)

    futures_data.sort_values(by='datetime',inplace=True)

    try:
        atm = base*np.round(futures_data[futures_data['datetime'] == entry_datetime]['open'].iloc[0]/base)
    except IndexError:
        print(f'ATM ERROR {data_date}')
        continue

    #get the nearest option expiry
    nearest_expiry = data[((data['instrument_type'] == 'CE')|(data['instrument_type'] == 'PE')) & (data['expiry_date'] >= data_date)]['expiry_date'].min()

    ce_strikes = data[(data['instrument_type'] == 'CE') & 
                      (data['instrument_name'] == instrument) & 
                      (data['expiry_date'] == nearest_expiry) & 
                      (data['datetime'] == entry_datetime)].copy()

    ce_strikes = ce_strikes.sort_values(by='close',ascending=False)
    ce_strike_price = ce_strikes[ce_strikes['close'] == ce_strikes[ce_strikes['close'] >= option_value]['close'].min()]['strike_price'].iloc[0]

    try:
        ce_data = data[(data['instrument_type'] == 'CE') & 
                       (data['instrument_name'] == instrument) & 
                       (data['expiry_date'] == nearest_expiry) & 
                       (data['strike_price'] == ce_strike_price)].copy()
        ce_symbol = ce_data['ticker'].iloc[0]
    except:
        while ce_strike_alternatives < 5:
            try:
                ce_data = data[(data['instrument_type'] == 'CE') & 
                               (data['instrument_name'] == instrument) & 
                               (data['expiry_date'] == nearest_expiry) & 
                               (data['strike_price'] == ce_strike_price+base)].copy()
                ce_symbol = ce_data['ticker'].iloc[0]
                ce_strike_alternatives+=1
                break
            except:
                continue

    #if ce data is retrieved reset the index
    ce_data.reset_index(drop=True,inplace=True)
    ce_data.sort_values(by='datetime',inplace=True)

    pe_strikes = data[(data['instrument_type'] == 'PE') & 
                        (data['instrument_name'] == instrument) & 
                        (data['expiry_date'] == nearest_expiry) & 
                        (data['datetime'] == entry_datetime)].copy()

    pe_strikes = pe_strikes.sort_values(by='open',ascending=False)
    pe_strike_price = pe_strikes[pe_strikes['close'] == pe_strikes[pe_strikes['close'] >= option_value]['close'].min()]['strike_price'].iloc[0]

    #get the pe data if some error comes, then shift the pe strike to atm-1
    #then also if the error comes, skip the day
    try:
        pe_data = data[(data['instrument_type'] == 'PE') & 
                        (data['instrument_name'] == instrument) & 
                        (data['expiry_date'] == nearest_expiry) & 
                        (data['strike_price'] == pe_strike_price)].copy()
        pe_symbol = pe_data['ticker'].iloc[0]
    except:
        while pe_strike_alternatives < 5:
            try:
                pe_data = data[(data['instrument_type'] == 'PE') & 
                               (data['instrument_name'] == instrument) & 
                               (data['expiry_date'] == nearest_expiry) & 
                               (data['strike_price'] == pe_strike_price+base)].copy()
                pe_symbol = pe_data['ticker'].iloc[0]
                pe_strike_alternatives+=1
                break
            except:
                continue

    #if pe data is retrieved reset the index
    pe_data.reset_index(drop=True,inplace=True)
    pe_data.sort_values(by='datetime',inplace=True)

    #rearrange the futures data
    futures_data = futures_data[['datetime','open','high','low','close']]
    futures_data.set_index('datetime',inplace=True)

    #rearrange the ce data
    ce_data = ce_data[['datetime','open','high','low','close']]
    ce_data = ce_data.rename(columns={'open':'ce_open',
                                        'high':'ce_high',
                                        'low':'ce_low',
                                        'close':'ce_close'})
    ce_data.drop_duplicates(subset='datetime',keep='first',inplace=True)
    ce_data.set_index('datetime',inplace=True)

    #rearrange the pe data
    pe_data = pe_data[['datetime','open','high','low','close']]
    pe_data = pe_data.rename(columns={'open':'pe_open',
                                        'high':'pe_high',
                                        'low':'pe_low',
                                        'close':'pe_close'})
    pe_data.drop_duplicates(subset='datetime',keep='first',inplace=True)
    pe_data.set_index('datetime',inplace=True)

    #combine the futures, ce and pe
    intraday_data = pd.concat([futures_data,ce_data,pe_data],axis=1)
    intraday_data.reset_index(inplace=True)
    intraday_data = intraday_data.ffill()

    #set the traded prices for futures, ce and pe
    traded_prices = intraday_data[intraday_data['datetime'] == entry_datetime].iloc[0]
    futures_entry_price = traded_prices[entry_offset]
    ce_entry_price = traded_prices[f'ce_{entry_offset}']
    pe_entry_price = traded_prices[f'pe_{entry_offset}']

    #if any of the traded prices comes as nan
    #try shifting the time to next minute until none of them come as nan
    #try for only 5 times
    while entry_minutes_alternative <= 5:

        if (np.isnan(futures_entry_price) == True)|(np.isnan(ce_entry_price) == True)|(np.isnan(pe_entry_price) == True):

            entry_datetime = entry_datetime + datetime.timedelta(minutes=entry_minutes_alternative)
            traded_prices = intraday_data[intraday_data['datetime'] == entry_datetime].iloc[0]
            futures_entry_price = traded_prices[entry_offset]
            ce_entry_price = traded_prices[f'ce_{entry_offset}']
            pe_entry_price = traded_prices[f'pe_{entry_offset}']
            entry_minutes_alternative+=1

        else:
            break

    #compute the ce and pe stop_loss
    ce_stop_loss = ce_entry_price + ce_entry_price*stop_loss_percentage
    pe_stop_loss = pe_entry_price + pe_entry_price*stop_loss_percentage
    
    try:
        #filter only the required data
        entry_time_index = intraday_data[intraday_data['datetime'] == entry_datetime].index[0]
        exit_time_index = intraday_data[intraday_data['datetime'] == exit_datetime].index[0]
        intraday_data = intraday_data[entry_time_index:exit_time_index+1]
    except IndexError:
        print(f'ERROR ON DATE {data_date}')
        continue
    
    ce_stop_loss_counter = 0
    pe_stop_loss_counter = 0
    ce_exit_datetime = ''
    pe_exit_datetime = ''
    ce_exit_price = 0
    pe_exit_price = 0
    ce_pnl = 0
    pe_pnl = 0
    pnl = 0

    intraday_data['ce_pnl'] = ce_entry_price - intraday_data['ce_close']
    intraday_data['pe_pnl'] = pe_entry_price - intraday_data['pe_close']
    intraday_data['pnl'] = intraday_data['ce_pnl'] + intraday_data['pe_pnl']

    intraday_data['ce_stop_loss_triggered'] = np.where(intraday_data[f'ce_{stop_loss_offset}'] >= ce_stop_loss,1,np.NaN)
    intraday_data['ce_stop_loss_triggered'].ffill(inplace=True)

    intraday_data['pe_stop_loss_triggered'] = np.where(intraday_data[f'pe_{stop_loss_offset}'] >= pe_stop_loss,1,np.NaN)
    intraday_data['pe_stop_loss_triggered'].ffill(inplace=True)

    try:
        ce_exit_data = intraday_data[intraday_data['ce_stop_loss_triggered'] == 1].iloc[0]
        ce_exit_price = round(ce_stop_loss/0.05)*0.05
        ce_stop_loss_counter = 1
    except:
        ce_exit_data = intraday_data.iloc[-1]
        ce_exit_price = ce_exit_data['ce_close']

    try:
        pe_exit_data = intraday_data[intraday_data['pe_stop_loss_triggered'] == 1].iloc[0]
        pe_exit_price = round(pe_stop_loss/0.05)*0.05
        pe_stop_loss_counter = 1
    except:
        pe_exit_data = intraday_data.iloc[-1]
        pe_exit_price = pe_exit_data['pe_close']

    ce_exit_datetime = ce_exit_data['datetime']
    pe_exit_datetime = pe_exit_data['datetime']

    ce_exit_time = ce_exit_datetime.time()
    pe_exit_time = pe_exit_datetime.time()

    trade_log = trade_log.append({'entry_date':data_date,
                                  'days_to_expiry':(nearest_expiry - data_date).days,
                                  'stop_loss':stop_loss_percentage*100,
                                  'ce_entry_time':entry_time,
                                  'ce_symbol':ce_symbol,
                                  'ce_entry':ce_entry_price,
                                  'ce_exit':ce_exit_price,
                                  'ce_exit_time':ce_exit_time,
                                  'ce_pnl':ce_entry_price-ce_exit_price,
                                  'pe_entry_time':entry_time,
                                  'pe_symbol':pe_symbol,
                                  'pe_entry':pe_entry_price,
                                  'pe_exit':pe_exit_price,
                                  'pe_exit_time':pe_exit_time,
                                  'pe_pnl':pe_entry_price-pe_exit_price},ignore_index=True)

    trade_log['pnl'] = trade_log['ce_pnl']+trade_log['pe_pnl']
    trade_log['weekday'] = trade_log['entry_date'].apply(lambda x: weekday_dict[x.weekday()])

FETCHED DATA FOR 2021-01-01
FETCHED DATA FOR 2021-01-04
FETCHED DATA FOR 2021-01-05
FETCHED DATA FOR 2021-01-06
FETCHED DATA FOR 2021-01-07
FETCHED DATA FOR 2021-01-08
FETCHED DATA FOR 2021-01-11
FETCHED DATA FOR 2021-01-12
FETCHED DATA FOR 2021-01-13
FETCHED DATA FOR 2021-01-14
FETCHED DATA FOR 2021-01-15
FETCHED DATA FOR 2021-01-18
FETCHED DATA FOR 2021-01-19
FETCHED DATA FOR 2021-01-20
FETCHED DATA FOR 2021-01-21
FETCHED DATA FOR 2021-01-22
FETCHED DATA FOR 2021-01-25
FETCHED DATA FOR 2021-01-27
FETCHED DATA FOR 2021-01-28
FETCHED DATA FOR 2021-01-29
FETCHED DATA FOR 2021-02-01
FETCHED DATA FOR 2021-02-02
FETCHED DATA FOR 2021-02-03
FETCHED DATA FOR 2021-02-04
FETCHED DATA FOR 2021-02-05
FETCHED DATA FOR 2021-02-08
FETCHED DATA FOR 2021-02-09
FETCHED DATA FOR 2021-02-10
FETCHED DATA FOR 2021-02-11
FETCHED DATA FOR 2021-02-12
FETCHED DATA FOR 2021-02-15
FETCHED DATA FOR 2021-02-16
FETCHED DATA FOR 2021-02-17
FETCHED DATA FOR 2021-02-18
FETCHED DATA FOR 2021-02-19
FETCHED DATA FOR 202

FETCHED DATA FOR 2022-03-09


In [92]:
trade_log['entry_price'] = trade_log['ce_entry'] + trade_log['pe_entry']
trade_log['exit_price'] = trade_log['ce_exit'] + trade_log['pe_exit']